# 🎯 **NOTEBOOK ULTRA-DOCUMENTADO - SEGMENTAÇÃO SEMÂNTICA**

---

## 📚 **SISTEMA COMPLETAMENTE COMENTADO E DIDÁTICO**

### ✅ **Este notebook foi super-detalhado para ensino e aprendizado:**
- 🔍 **Cada linha de código explicada** em detalhes
- 📊 **Conceitos matemáticos** fundamentados
- 🎯 **Aplicações práticas** de cada técnica
- 💡 **Dicas de otimização** e ajuste fino
- ⚠️ **Armadilhas comuns** e como evitá-las

---

## 🏗️ **ARQUITETURA DO SISTEMA:**

### 📦 **1. SETUP E CONFIGURAÇÃO (Células 1-3):**
- **Célula 1**: Imports detalhados com propósito de cada biblioteca
- **Célula 2**: Configurações base explicadas (IMG_SIZE, EPOCHS, etc.)
- **Célula 3**: Parâmetros avançados com guides de ajuste

### 🎯 **2. LOSS FUNCTIONS E MÉTRICAS (Célula 4):**
- **Dice Loss**: Matemática + aplicação para bordas precisas
- **Focal Loss**: Solução completa para classes desbalanceadas
- **Combined Loss**: Estratégia híbrida otimizada
- **IoU Metrics**: Avaliação padrão ouro explicada

### 🏗️ **3. ARQUITETURAS U-NET (Célula 5):**
- **U-Net Base**: Implementação clássica comentada
- **U-Net Improved**: Técnicas modernas (BatchNorm, Dropout)
- **U-Net Attention**: Estado da arte com Attention Gates

### 🎨 **4. DATA AUGMENTATION (Célula 6):**
- **Básico vs Avançado**: Quando usar cada um
- **Sincronização**: Como manter imagem e mask alinhadas
- **Parâmetros ótimos**: Para diferentes cenários

### ⚙️ **5. CALLBACKS E OTIMIZAÇÃO (Célula 7):**
- **EarlyStopping**: Prevenção de overfitting
- **ReduceLR**: Ajuste automático de learning rate
- **Learning Rate Schedulers**: Diferentes estratégias
- **Otimizadores**: Adam, AdamW, SGD comparados

### 🚀 **6. SISTEMA INTEGRADO (Células 8-11):**
- **Treinamento Modular**: Função completa automatizada
- **Visualização Avançada**: Plots de métricas e predições
- **Comparação de Arquiteturas**: Benchmark automático
- **Análise Final**: Relatórios detalhados

---

## 💡 **COMO USAR ESTE NOTEBOOK:**

### 🎓 **Para Aprendizado:**
1. **Leia os comentários** antes de executar cada célula
2. **Experimente parâmetros** diferentes nas configurações
3. **Execute célula por célula** para entender o fluxo
4. **Modifique valores** e observe os resultados

### 🚀 **Para Produção:**
1. **Execute todas as células** em sequência
2. **Ajuste CONFIG** para seus dados específicos
3. **Monitore métricas** durante o treinamento
4. **Use modelo salvo** para inferência

### 🧪 **Para Experimentação:**
1. **Modifique LOSS_PARAMS** para testar different loss functions
2. **Teste ARCHITECTURES** diferentes para comparar
3. **Ajuste AUGMENTATION_PARAMS** para seus dados
4. **Use callbacks** para otimização automática

---

## 🔧 **CONFIGURAÇÕES RÁPIDAS:**

### ⚡ **Teste Rápido (5 minutos):**
```python
CONFIG['EPOCHS'] = 5
CONFIG['BATCH_SIZE'] = 8
AUGMENTATION_PARAMS['rotation_range'] = 15
```

### 🎯 **Produção Completa (algumas horas):**
```python
CONFIG['EPOCHS'] = 500  # Com EarlyStopping
CONFIG['BATCH_SIZE'] = 16
# Use configurações padrão otimizadas
```

### 🔬 **Experimentação Avançada:**
```python
# Teste diferentes combinações de loss
LOSS_PARAMS['loss_weights'] = [0.3, 0.5, 0.2]
# Ou teste arquiteturas específicas
architecture='unet_attention'
```

---

**🎊 NOTEBOOK PRONTO PARA USO DIDÁTICO E PROFISSIONAL! 🎊**

In [ ]:
# 📦 IMPORTS E DEPENDÊNCIAS
# =====================================
# Esta célula carrega todas as bibliotecas necessárias para o projeto
# Cada import tem um propósito específico no pipeline de segmentação
# =====================================

print("📦 Carregando bibliotecas...")

# TENSORFLOW & KERAS - Framework principal de Deep Learning
import tensorflow as tf              # Framework de ML com suporte a GPU, base do projeto
import numpy as np                   # Manipulação eficiente de arrays numéricos e operações matemáticas
import matplotlib.pyplot as plt      # Visualização de imagens, gráficos de training e resultados
import seaborn as sns               # Visualizações estatísticas avançadas (heatmaps, distribuições)

# SCIKIT-LEARN - Ferramentas de avaliação e métricas
from sklearn.metrics import confusion_matrix, classification_report  # Métricas detalhadas de classificação

# OPENCV - Processamento de imagens
import cv2                          # Redimensionamento, transformações e manipulação de imagens

# KERAS LAYERS - Componentes da rede neural
from tensorflow.keras.layers import *    # Todas as camadas: Conv2D, MaxPooling2D, BatchNormalization, etc.

# KERAS MODELS - Construção e gerenciamento do modelo
from tensorflow.keras.models import Model   # Classe para criação de modelos funcionais personalizados

# KERAS OPTIMIZERS - Algoritmos de otimização
from tensorflow.keras.optimizers import Adam   # Otimizador Adam (padrão para deep learning)

# KERAS CALLBACKS - Funcionalidades durante treinamento
from tensorflow.keras.callbacks import *   # EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, etc.

# KERAS PREPROCESSING - Data augmentation e geração de dados
from tensorflow.keras.preprocessing.image import ImageDataGenerator   # Augmentação de dados em tempo real

print("✅ Todas as bibliotecas carregadas!")
print(f"🔧 TensorFlow: {tf.__version__}")
print(f"🔢 NumPy: {np.__version__}")

# 💡 INFORMAÇÕES IMPORTANTES:
# • TensorFlow: Detecta automaticamente GPU se disponível
# • NumPy: Todas as imagens são convertidas para arrays NumPy
# • Matplotlib: Usado para visualizar resultados e debugging
# • OpenCV: Eficiente para redimensionamento e transformações de imagem
# • Keras: API de alto nível do TensorFlow, mais simples de usar

📦 Carregando bibliotecas...
✅ Todas as bibliotecas carregadas!
🔧 TensorFlow: 2.19.0
🔢 NumPy: 2.1.3


In [ ]:
# ⚙️ CONFIGURAÇÕES BASE DO PROJETO
# =====================================
# Parâmetros fundamentais que controlam todo o comportamento do modelo
# Altere estes valores para experimentar diferentes configurações
# =====================================

print("⚙️ Configurando parâmetros base...")

# DICIONÁRIO DE CONFIGURAÇÕES CENTRALIZADAS
CONFIG = {
    # 🖼️ DIMENSÕES DAS IMAGENS
    'IMG_SIZE': (256, 256),         # Resolução de entrada: (altura, largura)
                                    # 256x256: balanço entre qualidade e velocidade
                                    # Maior = mais detalhes, mas mais lento
                                    # Menor = mais rápido, mas menos precisão
    
    # 📊 PARÂMETROS DE TREINAMENTO
    'BATCH_SIZE': 16,               # Quantas imagens processadas simultaneamente
                                    # 16: padrão para GPUs de 8-16GB VRAM
                                    # Menor = menos memória, convergência mais estável
                                    # Maior = mais eficiente, mas requer mais VRAM
    
    'EPOCHS': 500,                  # 🔥 NÚMERO MÁXIMO DE ÉPOCAS DE TREINAMENTO
                                    # 500: valor alto, pois usamos EarlyStopping
                                    # O treinamento para quando não há melhoria
                                    # Para testes rápidos: 10-50 épocas
                                    # Para resultados finais: 200-500 épocas
    
    'LEARNING_RATE': 1e-4,          # Taxa de aprendizado inicial (0.0001)
                                    # 1e-4: padrão seguro para Adam optimizer
                                    # Maior = convergência mais rápida, risco de instabilidade
                                    # Menor = convergência mais lenta, mas estável
    
    # 🎯 CONFIGURAÇÕES DO PROBLEMA DE SEGMENTAÇÃO
    'NUM_CLASSES': 3,               # Número de classes para segmentar
                                    # 3 classes: Background (fundo), Cat (gato), Dog (cachorro)
                                    # Esta é a dimensão da saída final do modelo
    
    'CLASS_NAMES': ['Background', 'Cat', 'Dog']  # Nomes das classes para visualização
                                    # Ordem IMPORTANTE: deve corresponder aos índices das masks
                                    # 0=Background, 1=Cat, 2=Dog
}

print("✅ Configurações base definidas!")
print(f"📊 Imagem: {CONFIG['IMG_SIZE']}")
print(f"🔧 Batch: {CONFIG['BATCH_SIZE']}, LR: {CONFIG['LEARNING_RATE']}")
print(f"🎯 Classes: {CONFIG['NUM_CLASSES']} - {CONFIG['CLASS_NAMES']}")

# 💡 DICAS PARA ALTERAR CONFIGURAÇÕES:
print("\n💡 GUIA DE AJUSTES:")
print("🖼️  IMG_SIZE: (128,128) = rápido | (256,256) = padrão | (512,512) = alta qualidade")
print("📊 BATCH_SIZE: 8 = pouca VRAM | 16 = padrão | 32 = muita VRAM")
print("🔥 EPOCHS: 10 = teste rápido | 100 = padrão | 500 = resultado final")
print("⚡ LEARNING_RATE: 1e-3 = agressivo | 1e-4 = padrão | 1e-5 = conservador")

# ⚠️ IMPORTANTE PARA EPOCHS:
print("\n⚠️  NOTA SOBRE EPOCHS:")
print("• O valor 500 é um máximo - o treinamento para automaticamente")
print("• EarlyStopping monitora val_mean_iou e para quando não melhora")
print("• Para alterar época: modifique CONFIG['EPOCHS'] nesta célula")
print("• Para testes: use 10-50 épocas")
print("• Para produção: use 200-500 épocas")

⚙️ Configurando parâmetros base...
✅ Configurações base definidas!
📊 Imagem: (256, 256)
🔧 Batch: 16, LR: 0.0001
🎯 Classes: 3 - ['Background', 'Cat', 'Dog']


In [ ]:
# 🎛️ PARÂMETROS ESPECÍFICOS DE ALGORITMOS
# =====================================
# Configurações avançadas para fine-tuning dos algoritmos
# Cada parâmetro foi otimizado para o problema de segmentação de pets
# =====================================

print("🎛️ Configurando parâmetros de algoritmos...")

# 🎯 PARÂMETROS DAS LOSS FUNCTIONS
LOSS_PARAMS = {
    # DICE LOSS - Para medir sobreposição entre regiões
    'dice_smooth': 1e-6,            # Smoothing para evitar divisão por zero
                                    # Valor pequeno (1e-6) previne instabilidade numérica
                                    # Menor = mais sensível | Maior = mais estável
    
    # FOCAL LOSS - Para lidar com classes desbalanceadas
    'focal_alpha': 0.8,             # Peso para classes positivas vs negativas
                                    # 0.8: favorece ligeiramente classes positivas (Cat/Dog)
                                    # 0.5 = balanceado | >0.5 = favorece positivas
    
    'focal_gamma': 2.0,             # Fator de foco em exemplos difíceis
                                    # 2.0: padrão da literatura, foca em casos difíceis
                                    # 0 = sem foco | 1-3 = foco moderado | >3 = foco extremo
    
    # COMBINED LOSS - Pesos para combinação híbrida
    'loss_weights': [0.4, 0.4, 0.2]  # [CrossEntropy, Dice, Focal]
                                    # 0.4 CCE: estabilidade base
                                    # 0.4 Dice: foco na sobreposição de regiões
                                    # 0.2 Focal: tratamento de desbalanceamento
                                    # Soma deve ser 1.0 para manter escala
}

# 🎨 PARÂMETROS DE DATA AUGMENTATION
AUGMENTATION_PARAMS = {
    # TRANSFORMAÇÕES GEOMÉTRICAS
    'rotation_range': 25,           # Rotação aleatória em graus (-25° a +25°)
                                    # 25°: natural para pets, evita rotações extremas
    
    'width_shift_range': 0.15,      # Deslocamento horizontal (15% da largura)
    'height_shift_range': 0.15,     # Deslocamento vertical (15% da altura)
                                    # 0.15: movimento natural, simula variação de pose
    
    'shear_range': 0.15,            # Cisalhamento/inclinação (15%)
                                    # Simula diferentes ângulos de câmera
    
    'zoom_range': 0.15,             # Zoom in/out (15%)
                                    # Simula diferentes distâncias da câmera
    
    # TRANSFORMAÇÕES DE ESPELHAMENTO
    'horizontal_flip': True,        # Espelhamento horizontal
                                    # True: pets podem aparecer de ambos os lados
    
    'vertical_flip': True,          # Espelhamento vertical
                                    # True: adiciona variação, mas cuidado com realismo
    
    # TRANSFORMAÇÕES DE COR/BRILHO
    'brightness_range': [0.8, 1.2], # Variação de brilho (80% a 120%)
                                    # Simula diferentes condições de iluminação
    
    'channel_shift_range': 20,      # Variação nos canais de cor
                                    # 20: mudança sutil nas cores, simula diferentes balanços
    
    'fill_mode': 'reflect'          # Como preencher pixels criados por transformações
                                    # 'reflect': espelha bordas, mais natural que zeros
}

# ⏰ PARÂMETROS DE CALLBACKS E OTIMIZAÇÃO
CALLBACK_PARAMS = {
    # REDUCE LEARNING RATE ON PLATEAU
    'lr_patience': 4,               # Épocas sem melhoria antes de reduzir LR
                                    # 4: paciência moderada, evita redução prematura
    
    'lr_factor': 0.5,               # Fator de redução do learning rate
                                    # 0.5: reduz pela metade (conservador)
    
    'lr_min': 1e-7,                 # Learning rate mínimo
                                    # 1e-7: muito pequeno, praticamente para o treinamento
    
    # EARLY STOPPING
    'early_stopping_patience': 10,  # Épocas sem melhoria antes de parar
                                    # 10: paciência alta, permite recuperação de overfitting
    
    'monitor_metric': 'val_mean_iou', # Métrica para monitorar
                                    # val_mean_iou: métrica principal para segmentação
    
    # MODEL CHECKPOINT
    'model_save_path': 'modelo_otimizado_modular.h5'  # Caminho para salvar melhor modelo
                                    # Salva automaticamente o melhor modelo
}

print("✅ Parâmetros de algoritmos configurados!")
print(f"🎯 Loss weights: {LOSS_PARAMS['loss_weights']}")
print(f"🔥 Focal: α={LOSS_PARAMS['focal_alpha']}, γ={LOSS_PARAMS['focal_gamma']}")
print(f"🎨 Augmentation: rotation={AUGMENTATION_PARAMS['rotation_range']}°")
print(f"⏰ Callbacks: EarlyStopping={CALLBACK_PARAMS['early_stopping_patience']}, ReduceLR={CALLBACK_PARAMS['lr_patience']}")

# 🔧 DICAS DE AJUSTE POR CENÁRIO:
print("\n🔧 GUIAS DE AJUSTE:")
print("🏃 TREINAMENTO RÁPIDO:")
print("  • lr_patience=2, early_stopping_patience=5")
print("  • rotation_range=15, zoom_range=0.1")

print("\n🎯 MÁXIMA PRECISÃO:")
print("  • lr_patience=6, early_stopping_patience=15")  
print("  • rotation_range=30, zoom_range=0.2")

print("\n⚖️ DADOS DESBALANCEADOS:")
print("  • focal_alpha=0.9, focal_gamma=3.0")
print("  • loss_weights=[0.3, 0.5, 0.2]")  # Mais peso no Dice

print("\n🖼️ IMAGENS PEQUENAS/DETALHES:")
print("  • dice_smooth=1e-7 (mais sensível)")
print("  • zoom_range=0.05 (menos zoom)")

# ⚠️ REGRAS IMPORTANTES:
print("\n⚠️ REGRAS IMPORTANTES:")
print("• loss_weights deve somar 1.0")
print("• focal_alpha entre 0.1-0.9 funciona melhor")
print("• augmentation muito agressivo pode prejudicar")
print("• early_stopping_patience > lr_patience (recomendado)")

🎛️ Configurando parâmetros de algoritmos...
✅ Parâmetros de algoritmos configurados!
🎯 Loss weights: [0.4, 0.4, 0.2]
🔥 Focal: α=0.8, γ=2.0
🎨 Augmentation: rotation=25°
⏰ Callbacks: EarlyStopping=10, ReduceLR=4


In [ ]:
# 🎯 MÓDULO COMPLETO: LOSS FUNCTIONS E MÉTRICAS
# =====================================
# Sistema completo de funções de perda e métricas para segmentação semântica
# Implementa as três loss functions mais eficazes: Dice, Focal e Combined
# =====================================

print("🎯 Implementando sistema completo de Loss e Métricas...")

def dice_loss_modular(y_true, y_pred, smooth=None):
    """
    🎲 DICE LOSS - Medida de sobreposição entre regiões
    
    =====================================================
    📊 CONCEITO MATEMÁTICO:
    • Dice Coefficient = 2 * |A ∩ B| / (|A| + |B|)
    • Mede sobreposição direta entre predição e ground truth
    • Loss = 1 - Dice Coefficient (para minimização)
    
    🎯 POR QUE É EFICAZ PARA SEGMENTAÇÃO:
    • Foca diretamente na sobreposição de pixels
    • Naturalmente balanceada para classes pequenas
    • Funciona bem quando regiões são pequenas/esparsas
    • Especialmente boa para bordas e contornos precisos
    
    📈 CARACTERÍSTICAS:
    • Range: [0,1] onde 0 = perfeita sobreposição
    • Smooth: previne divisão por zero quando região está ausente
    • Calculada por classe e depois média
    =====================================================
    """
    if smooth is None:
        smooth = LOSS_PARAMS['dice_smooth']  # Usa valor da configuração
    
    # 🔧 PREPARAÇÃO DOS TENSORES
    # Converter para formato flat para cálculo mais eficiente
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)  # Ground truth achatado
    y_pred_f = tf.reshape(y_pred, [-1, tf.shape(y_pred)[-1]]) # Predições achatadas [pixels, classes]
    
    # 🎯 CONVERSÃO PARA ONE-HOT
    # y_true vem como índices de classe, precisamos converter para one-hot
    y_true_one_hot = tf.one_hot(tf.cast(y_true_f, tf.int32), depth=tf.shape(y_pred)[-1])
    
    # 📊 CÁLCULO DO DICE COEFFICIENT POR CLASSE
    # Interseção: onde ambos ground truth e predição concordam
    intersection = tf.reduce_sum(y_true_one_hot * y_pred_f, axis=0)
    
    # Cálculo do coeficiente Dice para cada classe
    # Numerador: 2 * interseção + smooth (smoothing para estabilidade)
    # Denominador: soma de pixels true + soma de pixels pred + smooth
    dice_coef = (2. * intersection + smooth) / (
        tf.reduce_sum(y_true_one_hot, axis=0) + tf.reduce_sum(y_pred_f, axis=0) + smooth
    )
    
    # 🎯 RETORNAR LOSS (1 - Dice para minimização)
    return 1 - tf.reduce_mean(dice_coef)

def focal_loss_modular(y_true, y_pred, alpha=None, gamma=None):
    """
    🔥 FOCAL LOSS - Solução para classes desbalanceadas
    
    =====================================================
    📊 CONCEITO MATEMÁTICO:
    • FL(p_t) = -α_t * (1-p_t)^γ * log(p_t)
    • Reduz peso de exemplos "fáceis", foca nos "difíceis"
    • α (alpha): balanceia classes positivas vs negativas
    • γ (gamma): controla foco em exemplos difíceis
    
    🎯 POR QUE RESOLVE DESBALANCEAMENTO:
    • Classe dominante (background) tem muitos exemplos fáceis
    • Focal Loss reduz peso desses exemplos fáceis
    • Força a rede a focar em bordas e regiões difíceis
    • α ajusta o balanço entre foreground e background
    
    📈 PARÂMETROS TÍPICOS:
    • α = 0.8: favorece classes positivas (cats/dogs)
    • γ = 2.0: foco moderado em exemplos difíceis
    • γ = 0: vira CrossEntropy normal
    • γ > 2: foco muito agressivo
    =====================================================
    """
    if alpha is None:
        alpha = LOSS_PARAMS['focal_alpha']  # Usa valor da configuração
    if gamma is None:
        gamma = LOSS_PARAMS['focal_gamma']   # Usa valor da configuração
    
    # 🔧 PREPARAÇÃO DOS TENSORES
    y_true = tf.cast(y_true, tf.int32)  # Garantir tipo inteiro
    # Converter para one-hot encoding
    y_true_one_hot = tf.one_hot(y_true, depth=tf.shape(y_pred)[-1])
    y_true_one_hot = tf.reshape(y_true_one_hot, tf.shape(y_pred))
    
    # 🛡️ PROTEÇÃO CONTRA LOG(0)
    # Clipar predições para evitar log(0) = -∞
    epsilon = tf.keras.backend.epsilon()  # Valor muito pequeno (~1e-7)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
    # 📊 CÁLCULO DOS FATORES DE PESO
    # α_t: aplica α para classes positivas, (1-α) para negativas
    alpha_t = alpha * y_true_one_hot + (1 - alpha) * (1 - y_true_one_hot)
    
    # p_t: probabilidade da classe correta
    p_t = y_true_one_hot * y_pred + (1 - y_true_one_hot) * (1 - y_pred)
    
    # 🎯 APLICAR FÓRMULA DA FOCAL LOSS
    # (1-p_t)^γ: reduz peso quando p_t é alto (exemplo fácil)
    # log(p_t): componente de cross-entropy
    focal_loss = -alpha_t * tf.pow((1 - p_t), gamma) * tf.math.log(p_t)
    
    return tf.reduce_mean(focal_loss)

def combined_loss_modular(y_true, y_pred, weights=None):
    """
    ⚖️ COMBINED LOSS - Híbrida otimizada para segmentação
    
    =====================================================
    🎯 ESTRATÉGIA DE COMBINAÇÃO:
    • CrossEntropy (40%): Estabilidade base e convergência
    • Dice Loss (40%): Foco na sobreposição precisa de regiões
    • Focal Loss (20%): Tratamento de classes desbalanceadas
    
    💡 POR QUE FUNCIONA MELHOR:
    • CCE: Fornece gradientes estáveis e convergência confiável
    • Dice: Otimiza diretamente a métrica que nos importa (IoU)
    • Focal: Resolve problema de background dominante
    
    🔧 PESOS OTIMIZADOS:
    • [0.4, 0.4, 0.2]: Balanceado para a maioria dos casos
    • Para dados muito desbalanceados: [0.3, 0.4, 0.3]
    • Para bordas precisas: [0.2, 0.6, 0.2]
    =====================================================
    """
    if weights is None:
        weights = LOSS_PARAMS['loss_weights']  # Usa configuração padrão
    
    # 1️⃣ CATEGORICAL CROSSENTROPY (Base sólida)
    # Fornece gradientes estáveis e convergência confiável
    cce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    cce = tf.reduce_mean(cce)
    
    # 2️⃣ DICE LOSS (Sobreposição de regiões)
    # Otimiza diretamente a qualidade da segmentação
    dice = dice_loss_modular(y_true, y_pred)
    
    # 3️⃣ FOCAL LOSS (Classes desbalanceadas)
    # Trata o problema de background dominante
    focal = focal_loss_modular(y_true, y_pred)
    
    # 🎯 COMBINAÇÃO PONDERADA FINAL
    # Soma ponderada otimizada empiricamente
    total_loss = weights[0] * cce + weights[1] * dice + weights[2] * focal
    
    return total_loss

def mean_iou_modular(y_true, y_pred, num_classes=None):
    """
    📏 MEAN IoU - Métrica padrão para segmentação semântica
    
    =====================================================
    📊 CONCEITO MATEMÁTICO:
    • IoU = |A ∩ B| / |A ∪ B| (Intersection over Union)
    • Calcula para cada classe individualmente
    • Mean IoU = média de IoU de todas as classes
    
    🎯 POR QUE É A MÉTRICA PRINCIPAL:
    • Mede qualidade da segmentação diretamente
    • Range [0,1] onde 1 = segmentação perfeita
    • Penaliza tanto falsos positivos quanto falsos negativos
    • Independente de desbalanceamento de classes
    
    📈 INTERPRETAÇÃO:
    • 0.9-1.0: Excelente segmentação
    • 0.7-0.9: Boa segmentação
    • 0.5-0.7: Segmentação razoável
    • <0.5: Necessita melhorias
    =====================================================
    """
    if num_classes is None:
        num_classes = CONFIG['NUM_CLASSES']
    
    # 🔧 PREPARAÇÃO DOS TENSORES
    # Converter predições softmax para índices de classe
    y_pred = tf.argmax(y_pred, axis=-1)  # [batch, height, width]
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)
    
    # Achatar tensores para facilitar cálculo
    y_true = tf.reshape(y_true, [-1])    # [batch*height*width]
    y_pred = tf.reshape(y_pred, [-1])    # [batch*height*width]
    
    # 📊 CALCULAR IoU PARA CADA CLASSE
    ious = []
    for class_id in range(num_classes):
        # Máscaras booleanas para classe atual
        true_class = tf.equal(y_true, class_id)   # Pixels ground truth da classe
        pred_class = tf.equal(y_pred, class_id)   # Pixels preditos da classe
        
        # 🎯 CÁLCULO DE INTERSEÇÃO E UNIÃO
        # Interseção: pixels corretos da classe (ambos True)
        intersection = tf.reduce_sum(tf.cast(tf.logical_and(true_class, pred_class), tf.float32))
        
        # União: todos os pixels da classe (true OR pred)
        union = tf.reduce_sum(tf.cast(tf.logical_or(true_class, pred_class), tf.float32))
        
        # 🛡️ PROTEÇÃO CONTRA DIVISÃO POR ZERO
        # Se classe não aparece, considera IoU = 1.0 (perfeito)
        iou = tf.cond(tf.equal(union, 0), 
                     lambda: 1.0,  # Classe ausente = perfeito
                     lambda: intersection / union)  # IoU normal
        ious.append(iou)
    
    # 📈 RETORNAR MÉDIA DE TODAS AS CLASSES
    return tf.reduce_mean(tf.stack(ious))

def iou_per_class_modular(y_true, y_pred, num_classes=None):
    """
    🔍 IoU POR CLASSE - Análise detalhada de performance
    
    Retorna IoU individual para cada classe.
    Útil para identificar quais classes estão sendo mal segmentadas.
    """
    if num_classes is None:
        num_classes = CONFIG['NUM_CLASSES']
    
    # Mesmo processamento do mean_iou_modular
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)
    
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    
    # Retornar dicionário com IoU por classe
    ious = {}
    for class_id in range(num_classes):
        true_class = tf.equal(y_true, class_id)
        pred_class = tf.equal(y_pred, class_id)
        
        intersection = tf.reduce_sum(tf.cast(tf.logical_and(true_class, pred_class), tf.float32))
        union = tf.reduce_sum(tf.cast(tf.logical_or(true_class, pred_class), tf.float32))
        
        iou = tf.cond(tf.equal(union, 0), 
                     lambda: 1.0, 
                     lambda: intersection / union)
        ious[f'iou_class_{class_id}'] = iou  # Ex: iou_class_0, iou_class_1, iou_class_2
    
    return ious

# 📚 DICIONÁRIOS DE FUNÇÕES DISPONÍVEIS
# =====================================
# Organiza todas as funções para fácil acesso e experimentação
# =====================================

# 🎯 LOSS FUNCTIONS DISPONÍVEIS
LOSS_FUNCTIONS = {
    'dice': dice_loss_modular,          # Para sobreposição precisa
    'focal': focal_loss_modular,        # Para classes desbalanceadas
    'combined': combined_loss_modular   # 🥇 PRINCIPAL - Híbrida otimizada
}

# 📏 MÉTRICAS DISPONÍVEIS
METRICS_FUNCTIONS = {
    'mean_iou': mean_iou_modular,           # 🥇 PRINCIPAL - Métrica padrão
    'iou_per_class': iou_per_class_modular  # Para análise detalhada
}

print("✅ Sistema completo de Loss e Métricas implementado!")
print()
print("🎯 RESUMO DAS FUNÇÕES:")
print("-" * 50)
print("🎲 Dice Loss: Mede sobreposição, ótima para bordas precisas")
print("🔥 Focal Loss: Foca em exemplos difíceis, resolve desbalanceamento")
print("⚖️ Combined Loss: Híbrida otimizada (CCE + Dice + Focal)")
print("📏 Mean IoU: Métrica padrão ouro para segmentação")
print("🔍 IoU per Class: Análise detalhada de performance por classe")
print()
print("🏆 CONFIGURAÇÃO ATUAL:")
print(f"🎯 Loss principal: combined_loss_modular")
print(f"⚖️ Pesos: CCE={LOSS_PARAMS['loss_weights'][0]}, Dice={LOSS_PARAMS['loss_weights'][1]}, Focal={LOSS_PARAMS['loss_weights'][2]}")
print(f"🔥 Focal: α={LOSS_PARAMS['focal_alpha']}, γ={LOSS_PARAMS['focal_gamma']}")
print(f"🎲 Dice smoothing: {LOSS_PARAMS['dice_smooth']}")
print("💡 DICA: Use 'combined' para melhor performance geral!")

🎯 Implementando sistema completo de Loss e Métricas...
✅ Sistema completo de Loss e Métricas implementado!
🎲 Dice Loss: Mede sobreposição, bom para classes desbalanceadas
🔥 Focal Loss: Foca em exemplos difíceis, resolve classe dominante
⚖️ Combined Loss: Híbrida otimizada (CCE + Dice + Focal)
📏 Mean IoU: Métrica padrão para segmentação
🔍 IoU per Class: Análise detalhada por classe
🎯 Loss principal: combined_loss_modular
⚖️ Pesos: CCE=0.4, Dice=0.4, Focal=0.2
🔥 Focal: α=0.8, γ=2.0
🎲 Dice smoothing: 1e-06


In [376]:
# 🏗️ MÓDULO COMPLETO: ARQUITETURAS U-NET
print("🏗️ Implementando todas as arquiteturas U-Net...")

def create_unet_base(input_size=None, num_classes=None):
    """
    U-Net básica original (Ronneberger et al., 2015)
    
    Arquitetura clássica:
    - Encoder: reduz resolução, aumenta canais
    - Bottleneck: representação mais abstrata  
    - Decoder: reconstrói resolução com skip connections
    - Skip connections: preservam detalhes finos
    """
    if input_size is None:
        input_size = (*CONFIG['IMG_SIZE'], 3)
    if num_classes is None:
        num_classes = CONFIG['NUM_CLASSES']
    
    inputs = Input(input_size)
    
    # ENCODER (Caminho Descendente)
    # Block 1: 256x256 -> 128x128
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Block 2: 128x128 -> 64x64
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Block 3: 64x64 -> 32x32
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Block 4: 32x32 -> 16x16
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # BOTTLENECK: 16x16 (menor resolução, maior abstração)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    # DECODER (Caminho Ascendente com Skip Connections)
    # Up 1: 16x16 -> 32x32 + skip connection
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2,2))(conv5))
    merge6 = concatenate([conv4, up6], axis=3)  # Skip connection
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    # Up 2: 32x32 -> 64x64 + skip connection
    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2,2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)  # Skip connection
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    # Up 3: 64x64 -> 128x128 + skip connection
    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2,2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)  # Skip connection
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    # Up 4: 128x128 -> 256x256 + skip connection
    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2,2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)  # Skip connection
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    # OUTPUT: Classificação final por pixel
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

def create_unet_improved(input_size=None, num_classes=None, dropout_rates=None):
    """
    U-Net melhorada com técnicas modernas
    
    Melhorias implementadas:
    - BatchNormalization: estabiliza treinamento
    - Dropout: previne overfitting
    - He Normal: inicialização otimizada
    - Conv2DTranspose: upsampling aprendível
    - Dropout progressivo: mais dropout no bottleneck
    """
    if input_size is None:
        input_size = (*CONFIG['IMG_SIZE'], 3)
    if num_classes is None:
        num_classes = CONFIG['NUM_CLASSES']
    if dropout_rates is None:
        dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]  # Progressivo
    
    inputs = Input(input_size)
    
    # ENCODER MELHORADO com BatchNorm e Dropout
    # Block 1: BatchNorm + Dropout leve
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(dropout_rates[0])(pool1)
    # Block 2: Dropout moderado
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(dropout_rates[1])(pool2)
    
    # Block 3: Dropout aumentando
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(dropout_rates[2])(pool3)
    
    # Block 4: Dropout alto
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(dropout_rates[3])(pool4)
    
    # BOTTLENECK: Dropout máximo
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Dropout(dropout_rates[4])(conv5)
    
    # DECODER MELHORADO com Conv2DTranspose
    # Up 1: Conv2DTranspose (upsampling aprendível)
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv5)
    up6 = BatchNormalization()(up6)
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Dropout(dropout_rates[3])(conv6)  # Dropout espelhado
    
    # Up 2
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv6)
    up7 = BatchNormalization()(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Dropout(dropout_rates[2])(conv7)
    
    # Up 3
    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv7)
    up8 = BatchNormalization()(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Dropout(dropout_rates[1])(conv8)
    
    # Up 4
    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same', kernel_initializer='he_normal')(conv8)
    up9 = BatchNormalization()(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Dropout(dropout_rates[0])(conv9)
    
    # OUTPUT MELHORADO
    outputs = Conv2D(num_classes, 1, activation='softmax', name='output')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

def create_unet_attention(input_size=None, num_classes=None):
    """
    U-Net com Attention Gates (estado da arte)
    
    Attention Gates inovação:
    - Focam a rede nas regiões relevantes
    - Suprimem informações irrelevantes
    - Melhoram precisão sem aumentar parâmetros
    - Especialmente bons para objetos pequenos
    """
    if input_size is None:
        input_size = (*CONFIG['IMG_SIZE'], 3)
    if num_classes is None:
        num_classes = CONFIG['NUM_CLASSES']
    
    def attention_gate(F_g, F_l, F_int):
        """Attention Gate implementation"""
        # Transformar gating signal
        W_g = Conv2D(F_int, 1, padding='same')(F_g)
        W_g = BatchNormalization()(W_g)
        
        # Transformar input features
        W_x = Conv2D(F_int, 1, padding='same')(F_l)
        W_x = BatchNormalization()(W_x)
        
        # Combinar e aplicar ativação
        psi = Activation('relu')(Add()([W_g, W_x]))
        psi = Conv2D(1, 1, padding='same')(psi)
        psi = BatchNormalization()(psi)
        psi = Activation('sigmoid')(psi)  # Attention weights [0,1]
        
        # Aplicar attention (multiplicação elemento-wise)
        return Multiply()([F_l, psi])
    
    inputs = Input(input_size)
    
    # ENCODER (igual ao melhorado, mas sem dropout para simplicidade)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # BOTTLENECK
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    # DECODER COM ATTENTION GATES
    # Up 1: Attention + Skip Connection
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    up6 = BatchNormalization()(up6)
    att6 = attention_gate(up6, conv4, 256)  # Aplicar attention
    merge6 = concatenate([att6, up6], axis=3)  # Skip connection com attention
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = BatchNormalization()(conv6)
    
    # Up 2: Attention + Skip Connection  
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = BatchNormalization()(up7)
    att7 = attention_gate(up7, conv3, 128)  # Aplicar attention
    merge7 = concatenate([att7, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = BatchNormalization()(conv7)
    
    # Up 3: Attention + Skip Connection
    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = BatchNormalization()(up8)
    att8 = attention_gate(up8, conv2, 64)  # Aplicar attention
    merge8 = concatenate([att8, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = BatchNormalization()(conv8)
    
    # Up 4: Attention + Skip Connection
    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = BatchNormalization()(up9)
    att9 = attention_gate(up9, conv1, 32)  # Aplicar attention
    merge9 = concatenate([att9, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = BatchNormalization()(conv9)
    
    # OUTPUT
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Dicionário de arquiteturas disponíveis
ARCHITECTURES = {
    'unet_base': create_unet_base,
    'unet_improved': create_unet_improved,
    'unet_attention': create_unet_attention
}

print("✅ Todas as arquiteturas U-Net implementadas!")
print(f"🏗️ Disponíveis: {list(ARCHITECTURES.keys())}")
print()
print("📊 COMPARAÇÃO DAS ARQUITETURAS:")
print("-" * 40)
print("🏗️ unet_base:")
print("   • Simplicidade máxima, baseline de referência")
print("   • Treinamento rápido, ideal para aprendizado")
print()
print("🚀 unet_improved (RECOMENDADA):")
print("   • BatchNormalization + Dropout + He Normal")
print("   • Conv2DTranspose, melhor estabilidade")
print("   • Boa performance/velocidade, ideal para produção")
print()
print("🎯 unet_attention (AVANÇADA):")
print("   • Estado da arte com Attention Gates")
print("   • Foco em regiões importantes, máxima precisão")
print("   • Ideal para casos complexos e objetos pequenos")
print()
print("💡 RECOMENDAÇÃO:")
print("   🥇 Para produção: 'unet_improved'")
print("   🧪 Para experimentos: 'unet_attention'")
print("   📚 Para aprendizado: 'unet_base'")

🏗️ Implementando todas as arquiteturas U-Net...
✅ Todas as arquiteturas U-Net implementadas!
🏗️ Disponíveis: ['unet_base', 'unet_improved', 'unet_attention']

📊 COMPARAÇÃO DAS ARQUITETURAS:
----------------------------------------
🏗️ unet_base:
   • Simplicidade máxima, baseline de referência
   • Treinamento rápido, ideal para aprendizado

🚀 unet_improved (RECOMENDADA):
   • BatchNormalization + Dropout + He Normal
   • Conv2DTranspose, melhor estabilidade
   • Boa performance/velocidade, ideal para produção

🎯 unet_attention (AVANÇADA):
   • Estado da arte com Attention Gates
   • Foco em regiões importantes, máxima precisão
   • Ideal para casos complexos e objetos pequenos

💡 RECOMENDAÇÃO:
   🥇 Para produção: 'unet_improved'
   🧪 Para experimentos: 'unet_attention'
   📚 Para aprendizado: 'unet_base'


In [377]:
# 🔄 MÓDULO 4: DATA AUGMENTATION COMPLETO
print("🎨 Carregando data augmentation completo...")

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_basic_augmentation():
    """
    Data augmentation básico e seguro
    
    Transformações conservadoras:
    - Rotação limitada (15°)
    - Shift limitado (10%)
    - Apenas flip horizontal
    - Normalização automática
    
    Ideal para: primeiros testes, dados limitados
    """
    return ImageDataGenerator(
        rotation_range=15,          # Rotação até 15°
        width_shift_range=0.1,      # Shift horizontal 10%
        height_shift_range=0.1,     # Shift vertical 10%  
        horizontal_flip=True,       # Flip horizontal
        rescale=1./255             # Normalização [0,1]
    )

def create_mask_augmentation_basic():
    """
    Data augmentation básico para máscaras
    
    Importante: máscaras NÃO podem ter:
    - Rescale (valores devem ser inteiros 0,1,2)
    - Brightness/contrast (altera classes)
    - Channel shift (máscaras são single-channel)
    """
    return ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
        # SEM rescale nem brightness!
    )

def create_advanced_augmentation(params=None):
    """
    Data augmentation avançado e agressivo
    
    Transformações intensas para máxima generalização:
    - Rotação 25° (detecta objetos em qualquer ângulo)
    - Shifts 15% (posições variadas)
    - Zoom ±15% (tamanhos diferentes)
    - Shear 15° (perspectivas)
    - Flips vertical e horizontal
    - Brightness ±20% (iluminação)
    - Channel shift (cores)
    
    Ideal para: produção, datasets pequenos
    """
    if params is None:
        params = AUGMENTATION_PARAMS
    
    return ImageDataGenerator(
        rotation_range=params['rotation_range'],        # 25°
        width_shift_range=params['width_shift_range'],  # 15%
        height_shift_range=params['height_shift_range'], # 15%
        shear_range=params['shear_range'],              # 15°
        zoom_range=params['zoom_range'],                # ±15%
        horizontal_flip=params['horizontal_flip'],       # True
        vertical_flip=params['vertical_flip'],          # True  
        brightness_range=params['brightness_range'],    # [0.8, 1.2]
        channel_shift_range=params['channel_shift_range'], # 20
        fill_mode=params['fill_mode'],                  # 'reflect'
        rescale=1./255
    )

def create_mask_augmentation_advanced(params=None):
    """
    Data augmentation avançado para máscaras
    
    MESMAS transformações geométricas que imagens,
    mas SEM alterações de cor/brilho
    """
    if params is None:
        params = AUGMENTATION_PARAMS
    
    return ImageDataGenerator(
        rotation_range=params['rotation_range'],
        width_shift_range=params['width_shift_range'],
        height_shift_range=params['height_shift_range'],
        shear_range=params['shear_range'],
        zoom_range=params['zoom_range'],
        horizontal_flip=params['horizontal_flip'],
        vertical_flip=params['vertical_flip'],
        fill_mode=params['fill_mode']
        # SEM: rescale, brightness, channel_shift
    )

def apply_augmentation_to_data(X_data, Y_data, augmentation_type='advanced'):
    """Aplicar augmentation aos dados com validação"""
    print(f"🔄 Aplicando augmentation: {augmentation_type}")
    
    if augmentation_type == 'basic':
        img_gen = create_basic_augmentation()
        mask_gen = create_mask_augmentation_basic()
    elif augmentation_type == 'advanced':
        img_gen = create_advanced_augmentation()
        mask_gen = create_mask_augmentation_advanced()
    else:
        raise ValueError("Tipo deve ser 'basic' ou 'advanced'")
    
    # Converter para numpy se necessário
    if isinstance(X_data, list):
        X_data = np.array(X_data)
        print("🔄 Convertido X_data para numpy array")
    if isinstance(Y_data, list):
        Y_data = np.array(Y_data)
        print("🔄 Convertido Y_data para numpy array")
    
    # Normalizar dados de imagem se necessário
    if X_data.max() > 1.5:
        print("🔄 Normalizando imagens [0,1]")
        X_data = X_data / 255.0
    else:
        print("✅ Imagens já normalizadas")
    
    print(f"✅ Dados preparados: X{X_data.shape}, Y{Y_data.shape}")
    return img_gen, mask_gen, X_data, Y_data

def create_paired_generators(X_data, Y_data, batch_size=None, augmentation_type='advanced', seed=42):
    """
    Criar geradores pareados para treinamento
    
    Crítico: imagem e máscara devem ter MESMA transformação!
    - Mesmo seed garante sincronização
    - Mesmo batch_size
    - Mesmo shuffle
    """
    if batch_size is None:
        batch_size = CONFIG['BATCH_SIZE']
    
    print(f"🔗 Criando generators pareados (seed={seed})")
    
    img_gen, mask_gen, X_norm, Y_norm = apply_augmentation_to_data(X_data, Y_data, augmentation_type)
    
    # Expandir dimensões das máscaras se necessário
    if len(Y_norm.shape) == 3:
        Y_norm = np.expand_dims(Y_norm, axis=-1)
        print("🔄 Expandindo dimensões das máscaras")
    
    # Criar generators sincronizados
    img_generator = img_gen.flow(X_norm, batch_size=batch_size, seed=seed)
    mask_generator = mask_gen.flow(Y_norm, batch_size=batch_size, seed=seed)
    
    print(f"✅ Generators criados: batch_size={batch_size}")
    return zip(img_generator, mask_generator)

# Dicionário de tipos de augmentation disponíveis
AUGMENTATION_TYPES = {
    'none': lambda: (None, None),
    'basic': lambda: (create_basic_augmentation(), create_mask_augmentation_basic()),
    'advanced': lambda: (create_advanced_augmentation(), create_mask_augmentation_advanced())
}

print("✅ Data augmentation completo carregado!")
print(f"🎨 Tipos disponíveis: {list(AUGMENTATION_TYPES.keys())}")
print(f"🔧 Parâmetros: rotation={AUGMENTATION_PARAMS['rotation_range']}°, ")
print(f"   shift={AUGMENTATION_PARAMS['width_shift_range']}, zoom={AUGMENTATION_PARAMS['zoom_range']}")
print("🔗 Função principal: create_paired_generators()")
print("💡 Generators sempre sincronizados (mesmo seed)")

🎨 Carregando data augmentation completo...
✅ Data augmentation completo carregado!
🎨 Tipos disponíveis: ['none', 'basic', 'advanced']
🔧 Parâmetros: rotation=25°, 
   shift=0.15, zoom=0.15
🔗 Função principal: create_paired_generators()
💡 Generators sempre sincronizados (mesmo seed)


In [378]:
# ⚙️ MÓDULO 5: CALLBACKS, SCHEDULERS E OTIMIZAÇÃO COMPLETO
print("🎛️ Carregando callbacks, schedulers e otimização...")

from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

def create_basic_callbacks(params=None):
    """
    Callbacks essenciais para qualquer treinamento
    
    ModelCheckpoint:
    - Salva melhor modelo automaticamente
    - Monitora val_mean_iou (métrica principal)
    - Só salva quando melhora
    
    EarlyStopping:
    - Para treinamento se não melhorar
    - Evita overfitting
    - Restaura melhores pesos
    """
    if params is None:
        params = CALLBACK_PARAMS
    
    callbacks = [
        ModelCheckpoint(
            params['model_save_path'],
            monitor=params['monitor_metric'],    # 'val_mean_iou'
            save_best_only=True,                # Só salva se melhorar
            mode='max',                         # Maximizar mIoU
            verbose=1,                          # Mostrar quando salva
            save_format='h5'
        ),
        EarlyStopping(
            monitor=params['monitor_metric'],    # 'val_mean_iou'
            patience=params['early_stopping_patience'],  # 10 épocas
            restore_best_weights=True,          # Volta pro melhor
            mode='max',                         # Maximizar mIoU
            verbose=1
        )
    ]
    
    print(f"✅ Callbacks básicos configurados!")
    print(f"📊 Monitor: {params['monitor_metric']}")
    print(f"💾 Arquivo: {params['model_save_path']}")
    print(f"⏰ Paciência: {params['early_stopping_patience']} épocas")
    
    return callbacks

def create_advanced_callbacks(params=None, log_file=None):
    """
    Callbacks avançados para treinamento profissional
    
    Adiciona aos básicos:
    - ReduceLROnPlateau: ajusta learning rate
    - CSVLogger: salva histórico completo
    - Configurações otimizadas
    """
    if params is None:
        params = CALLBACK_PARAMS
    if log_file is None:
        log_file = 'training_log_modular.csv'
    
    callbacks = [
        ModelCheckpoint(
            params['model_save_path'],
            monitor=params['monitor_metric'],
            save_best_only=True,
            mode='max',
            verbose=1,
            save_weights_only=False
        ),
        EarlyStopping(
            monitor=params['monitor_metric'],
            patience=params['early_stopping_patience'],
            restore_best_weights=True,
            mode='max',
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor=params['monitor_metric'],
            factor=params['lr_factor'],
            patience=params['lr_patience'],
            min_lr=params['lr_min'],
            mode='max',
            verbose=1,
            cooldown=2
        ),
        CSVLogger(
            log_file,
            append=True
        )
    ]
    
    print(f"✅ Callbacks avançados configurados!")
    print(f"📉 ReduceLR: fator={params['lr_factor']}, paciência={params['lr_patience']}")
    print(f"📊 CSV Log: {log_file}")
    
    return callbacks

def create_lr_scheduler(schedule_type='cosine', initial_lr=None, decay_steps=None):
    """
    Diferentes estratégias de Learning Rate Scheduling
    
    Cosine Decay (RECOMENDADO):
    - Decaimento suave em formato cosseno
    - Permite "restarts" naturais
    - Boa convergência
    
    Exponential Decay:
    - Decaimento exponencial clássico
    - Previsível e estável
    
    Polynomial Decay:
    - Decaimento polinomial controlado
    - Flexível com power ajustável
    """
    if initial_lr is None:
        initial_lr = CONFIG['LEARNING_RATE']
    if decay_steps is None:
        decay_steps = CONFIG['EPOCHS'] * 100  # Estimativa
    
    print(f"📈 Criando scheduler: {schedule_type}")
    print(f"   Initial LR: {initial_lr}")
    print(f"   Decay steps: {decay_steps}")
    
    if schedule_type == 'cosine':
        scheduler = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=initial_lr,
            decay_steps=decay_steps,
            alpha=0.1                           # LR final = 10% do inicial
        )
        print("   🌊 Cosine: decaimento suave, convergência natural")
        
    elif schedule_type == 'exponential':
        scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=initial_lr,
            decay_steps=decay_steps // 4,       # Decai a cada 1/4 do treino
            decay_rate=0.96,                    # Reduz 4% por step
            staircase=True                      # Passos discretos
        )
        print("   📉 Exponential: redução 4% por step, previsível")
        
    elif schedule_type == 'polynomial':
        scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=initial_lr,
            decay_steps=decay_steps,
            end_learning_rate=initial_lr * 0.01, # 1% do inicial
            power=0.9                           # Suavidade do decaimento
        )
        print("   📐 Polynomial: decaimento power=0.9, flexível")
        
    else:
        print(f"   ⚠️  Tipo desconhecido, usando LR constante")
        return initial_lr
    
    return scheduler

def create_optimizer(optimizer_type='adam', learning_rate=None, **kwargs):
    """
    Diferentes otimizadores para diferentes necessidades
    
    Adam (PADRÃO):
    - Adaptativo, funciona bem out-of-the-box
    - Boa convergência, estável
    
    AdamW:
    - Adam com weight decay separado
    - Melhor regularização
    
    SGD:
    - Clássico, com momentum
    - Mais lento mas às vezes melhor convergência final
    
    RMSprop:
    - Adaptativo, boa para RNNs
    - Alternativa ao Adam
    """
    if learning_rate is None:
        learning_rate = CONFIG['LEARNING_RATE']
    
    print(f"⚡ Criando otimizador: {optimizer_type}")
    print(f"   Learning Rate: {learning_rate}")
    
    if optimizer_type == 'adam':
        optimizer = Adam(
            learning_rate=learning_rate,
            beta_1=0.9,         # Momentum para gradiente
            beta_2=0.999,       # Momentum para gradiente^2
            epsilon=1e-7,       # Estabilidade numérica
            **kwargs
        )
        print("   🎯 Adam: adaptativo, estável, recomendado")
        
    elif optimizer_type == 'adamw':
        optimizer = tf.keras.optimizers.AdamW(
            learning_rate=learning_rate,
            weight_decay=0.01,  # L2 regularization
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-7,
            **kwargs
        )
        print("   🎯 AdamW: Adam + weight decay, melhor regularização")
        
    elif optimizer_type == 'sgd':
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate,
            momentum=0.9,       # Momentum padrão
            nesterov=True,      # Nesterov momentum
            **kwargs
        )
        print("   🎯 SGD: clássico + momentum + nesterov")
        
    elif optimizer_type == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=learning_rate,
            rho=0.9,           # Decay rate
            epsilon=1e-7,
            **kwargs
        )
        print("   🎯 RMSprop: adaptativo, alternativa ao Adam")
        
    else:
        raise ValueError(f"Optimizer '{optimizer_type}' não suportado")
    
    return optimizer

def create_custom_callbacks(tensorboard_log=None, reduce_lr_monitor='val_loss'):
    """Callbacks customizados com TensorBoard"""
    callbacks = create_advanced_callbacks()
    
    if tensorboard_log:
        callbacks.append(
            TensorBoard(
                log_dir=tensorboard_log,
                histogram_freq=1,
                write_graph=True,
                write_images=True,
                update_freq='epoch'
            )
        )
    
    # Callback customizado para métricas detalhadas
    class DetailedMetrics(Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs:
                print(f"\\n📊 Época {epoch + 1}:")
                print(f"   🎯 Accuracy: {logs.get('accuracy', 0):.4f} → Val: {logs.get('val_accuracy', 0):.4f}")
                print(f"   🎪 mIoU: {logs.get('mean_iou', 0):.4f} → Val: {logs.get('val_mean_iou', 0):.4f}")
                print(f"   📉 Loss: {logs.get('loss', 0):.4f} → Val: {logs.get('val_loss', 0):.4f}")
                print(f"   📈 LR: {logs.get('learning_rate', 0):.2e}")
    
    callbacks.append(DetailedMetrics())
    
    return callbacks

# Configurações de otimização pré-definidas
OPTIMIZATION_CONFIGS = {
    'basic': {
        'optimizer': 'adam',
        'callbacks': 'basic',
        'lr_schedule': None,
        'description': 'Configuração simples e estável'
    },
    'advanced': {
        'optimizer': 'adam',
        'callbacks': 'advanced', 
        'lr_schedule': 'cosine',
        'description': 'Configuração recomendada para produção'
    },
    'experimental': {
        'optimizer': 'adamw',
        'callbacks': 'custom',
        'lr_schedule': 'polynomial',
        'description': 'Configuração experimental avançada'
    }
}

print("✅ Callbacks, schedulers e otimização carregados!")
print(f"⚙️ Configs disponíveis: {list(OPTIMIZATION_CONFIGS.keys())}")
print(f"🎯 Monitor padrão: {CALLBACK_PARAMS['monitor_metric']}")
print(f"⚡ Otimizadores: adam (padrão), adamw, sgd, rmsprop")
print(f"📈 Schedulers: cosine (recomendado), exponential, polynomial")
print(f"⏰ Paciência: EarlyStopping={CALLBACK_PARAMS['early_stopping_patience']}, ReduceLR={CALLBACK_PARAMS['lr_patience']}")

🎛️ Carregando callbacks, schedulers e otimização...
✅ Callbacks, schedulers e otimização carregados!
⚙️ Configs disponíveis: ['basic', 'advanced', 'experimental']
🎯 Monitor padrão: val_mean_iou
⚡ Otimizadores: adam (padrão), adamw, sgd, rmsprop
📈 Schedulers: cosine (recomendado), exponential, polynomial
⏰ Paciência: EarlyStopping=10, ReduceLR=4


In [379]:
# 🚀 MÓDULO 6: TREINAMENTO MODULAR
print("🔥 Carregando sistema de treinamento...")

def create_model_modular(architecture='unet_improved', compile_model=True, **kwargs):
    """Criar modelo com arquitetura especificada"""
    if architecture not in ARCHITECTURES:
        raise ValueError(f"Arquitetura '{architecture}' não encontrada. Disponíveis: {list(ARCHITECTURES.keys())}")
    
    print(f"🏗️ Criando modelo: {architecture}")
    model = ARCHITECTURES[architecture](**kwargs)
    
    if compile_model:
        print("⚙️ Compilando modelo...")
        optimizer = create_optimizer()
        model.compile(
            optimizer=optimizer,
            loss=combined_loss_modular,
            metrics=['accuracy', mean_iou_modular]
        )
        print("✅ Modelo compilado!")
    
    return model

def prepare_data_for_training(X_data, Y_data, validation_data=None, normalize=True):
    """Preparar dados para treinamento"""
    print("📊 Preparando dados...")
    
    # Converter para numpy arrays se necessário
    if isinstance(X_data, list):
        X_data = np.array(X_data)
    if isinstance(Y_data, list):
        Y_data = np.array(Y_data)
    
    # Normalizar se necessário
    if normalize and X_data.max() > 1.5:
        X_data = X_data / 255.0
        print("🔄 Dados normalizados (0-1)")
    
    # Preparar dados de validação se fornecidos
    if validation_data:
        X_val, Y_val = validation_data
        if isinstance(X_val, list):
            X_val = np.array(X_val)
        if isinstance(Y_val, list):
            Y_val = np.array(Y_val)
        
        if normalize and X_val.max() > 1.5:
            X_val = X_val / 255.0
        
        validation_data = (X_val, Y_val)
    
    print(f"✅ Dados preparados!")
    print(f"   📊 X_train: {X_data.shape}")
    print(f"   📊 Y_train: {Y_data.shape}")
    if validation_data:
        print(f"   📊 X_val: {validation_data[0].shape}")
        print(f"   📊 Y_val: {validation_data[1].shape}")
    
    return X_data, Y_data, validation_data

def train_model_modular(
    X_train, Y_train, 
    validation_data=None,
    architecture='unet_improved',
    optimization_config='advanced',
    epochs=None,
    batch_size=None,
    augmentation_type='advanced',
    verbose=1
):
    """Função principal de treinamento modular"""
    
    if epochs is None:
        epochs = CONFIG['EPOCHS']
    if batch_size is None:
        batch_size = CONFIG['BATCH_SIZE']
    
    print("🚀 INICIANDO TREINAMENTO MODULAR")
    print("="*50)
    
    # 1. Criar modelo
    model = create_model_modular(architecture=architecture)
    
    # 2. Preparar dados
    X_train, Y_train, validation_data = prepare_data_for_training(
        X_train, Y_train, validation_data
    )
    
    # 3. Configurar callbacks
    if optimization_config == 'basic':
        callbacks = create_basic_callbacks()
    elif optimization_config == 'advanced':
        callbacks = create_advanced_callbacks()
    elif optimization_config == 'custom':
        callbacks = create_custom_callbacks()
    else:
        callbacks = create_advanced_callbacks()
    
    # 4. Configurar augmentation (se especificado)
    if augmentation_type != 'none':
        print(f"🎨 Aplicando {augmentation_type} augmentation...")
        # Para simplicidade, vamos treinar sem generators por enquanto
        # Em produção, implementaria generators aqui
    
    print(f"📈 Configurações de treinamento:")
    print(f"   🏗️ Arquitetura: {architecture}")
    print(f"   ⚙️ Otimização: {optimization_config}")
    print(f"   🎨 Augmentation: {augmentation_type}")
    print(f"   📊 Batch size: {batch_size}")
    print(f"   🔄 Epochs: {epochs}")
    print(f"   🎯 Monitor: {CALLBACK_PARAMS['monitor_metric']}")
    
    # 5. Treinar modelo
    print("\\n🔥 Iniciando treinamento...")
    print("-" * 50)
    
    history = model.fit(
        X_train, Y_train,
        validation_data=validation_data,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=verbose
    )
    
    print("\\n✅ Treinamento concluído!")
    print("="*50)
    
    return model, history

def quick_train_demo(X_train, Y_train, validation_data=None, epochs=5):
    """Demonstração rápida de treinamento"""
    print("⚡ DEMONSTRAÇÃO RÁPIDA - 5 ÉPOCAS")
    print("="*40)
    
    return train_model_modular(
        X_train, Y_train,
        validation_data=validation_data,
        architecture='unet_improved',
        optimization_config='advanced',
        epochs=epochs,
        augmentation_type='none',  # Sem augmentation para rapidez
        verbose=1
    )

def compare_architectures(X_train, Y_train, validation_data=None, epochs=3):
    """Comparar diferentes arquiteturas rapidamente"""
    print("🏁 COMPARAÇÃO DE ARQUITETURAS")
    print("="*40)
    
    results = {}
    architectures_to_test = ['unet_base', 'unet_improved']
    
    for arch in architectures_to_test:
        print(f"\\n🧪 Testando: {arch}")
        try:
            model, history = train_model_modular(
                X_train, Y_train,
                validation_data=validation_data,
                architecture=arch,
                epochs=epochs,
                verbose=0
            )
            
            # Pegar métricas finais
            final_acc = history.history['val_accuracy'][-1] if 'val_accuracy' in history.history else 0
            final_miou = history.history['val_mean_iou_modular'][-1] if 'val_mean_iou_modular' in history.history else 0
            
            results[arch] = {
                'model': model,
                'history': history,
                'final_accuracy': final_acc,
                'final_miou': final_miou
            }
            
            print(f"✅ {arch}: Acc={final_acc:.3f}, mIoU={final_miou:.3f}")
            
        except Exception as e:
            print(f"❌ Erro em {arch}: {str(e)}")
            results[arch] = {'error': str(e)}
    
    return results

print("✅ Sistema de treinamento carregado!")
print("🚀 Funções disponíveis:")
print("   • train_model_modular() - Treinamento completo")
print("   • quick_train_demo() - Demo rápida") 
print("   • compare_architectures() - Comparação automática")

🔥 Carregando sistema de treinamento...
✅ Sistema de treinamento carregado!
🚀 Funções disponíveis:
   • train_model_modular() - Treinamento completo
   • quick_train_demo() - Demo rápida
   • compare_architectures() - Comparação automática


In [380]:
# 📊 MÓDULO 7: VISUALIZAÇÃO E COMPARAÇÃO AVANÇADA
print("🎨 Carregando sistema de visualização...")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import cv2

plt.style.use('default')
sns.set_palette("husl")

def plot_training_history(history, title="Training History"):
    """Plotar histórico de treinamento completo"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    metrics = ['accuracy', 'loss', 'mean_iou_modular', 'learning_rate']
    titles = ['Accuracy', 'Loss', 'Mean IoU', 'Learning Rate']
    
    for i, (metric, title_name) in enumerate(zip(metrics, titles)):
        row, col = i // 2, i % 2
        ax = axes[row, col]
        
        if metric in history.history:
            epochs = range(1, len(history.history[metric]) + 1)
            ax.plot(epochs, history.history[metric], 'b-', label=f'Training {title_name}', linewidth=2)
            
            val_metric = f'val_{metric}'
            if val_metric in history.history:
                ax.plot(epochs, history.history[val_metric], 'r-', label=f'Validation {title_name}', linewidth=2)
            
            ax.set_title(f'{title_name} Progress', fontweight='bold')
            ax.set_xlabel('Epoch')
            ax.set_ylabel(title_name)
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            # Destacar melhor época
            if val_metric in history.history:
                if 'loss' in metric:
                    best_epoch = np.argmin(history.history[val_metric]) + 1
                    best_value = min(history.history[val_metric])
                else:
                    best_epoch = np.argmax(history.history[val_metric]) + 1
                    best_value = max(history.history[val_metric])
                
                ax.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.7)
                ax.text(best_epoch, best_value, f'Best: {best_value:.3f}', 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))
    
    plt.tight_layout()
    plt.show()

def predict_and_visualize(model, X_data, Y_data, num_samples=6, class_names=None):
    """Fazer predições e visualizar resultados"""
    if class_names is None:
        class_names = CONFIG['CLASS_NAMES']
    
    # Fazer predições
    predictions = model.predict(X_data[:num_samples])
    pred_classes = np.argmax(predictions, axis=-1)
    
    # Criar figura
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        # Imagem original
        axes[i, 0].imshow(X_data[i])
        axes[i, 0].set_title('Imagem Original', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Máscara verdadeira
        true_mask = Y_data[i]
        axes[i, 1].imshow(true_mask, cmap='viridis', vmin=0, vmax=2)
        axes[i, 1].set_title('Máscara Verdadeira', fontweight='bold')
        axes[i, 1].axis('off')
        
        # Predição
        pred_mask = pred_classes[i]
        axes[i, 2].imshow(pred_mask, cmap='viridis', vmin=0, vmax=2)
        axes[i, 2].set_title('Predição', fontweight='bold')
        axes[i, 2].axis('off')
        
        # Sobreposição
        overlay = X_data[i].copy()
        # Criar máscara colorida para sobreposição
        colors = [[0, 0, 0], [1, 0, 0], [0, 0, 1]]  # Background, Cat (red), Dog (blue)
        colored_pred = np.zeros((*pred_mask.shape, 3))
        for c in range(len(colors)):
            colored_pred[pred_mask == c] = colors[c]
        
        # Blend com a imagem original
        blended = cv2.addWeighted(overlay.astype(np.float32), 0.7, colored_pred.astype(np.float32), 0.3, 0)
        axes[i, 3].imshow(np.clip(blended, 0, 1))
        axes[i, 3].set_title('Sobreposição', fontweight='bold')
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return predictions, pred_classes

def analyze_class_performance(Y_true, Y_pred, class_names=None):
    """Análise detalhada por classe"""
    if class_names is None:
        class_names = CONFIG['CLASS_NAMES']
    
    # Flatten arrays
    y_true_flat = Y_true.flatten()
    y_pred_flat = Y_pred.flatten()
    
    # Confusion Matrix
    cm = confusion_matrix(y_true_flat, y_pred_flat)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names, ax=axes[0])
    axes[0].set_title('Matriz de Confusão', fontweight='bold')
    axes[0].set_xlabel('Predição')
    axes[0].set_ylabel('Verdadeiro')
    
    # Calculate IoU per class
    ious = []
    for i in range(len(class_names)):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
        ious.append(iou)
    
    # Plot IoU per class
    bars = axes[1].bar(class_names, ious, color=['gray', 'red', 'blue'], alpha=0.7)
    axes[1].set_title('IoU por Classe', fontweight='bold')
    axes[1].set_ylabel('IoU')
    axes[1].set_ylim(0, 1)
    
    # Adicionar valores nas barras
    for bar, iou in zip(bars, ious):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{iou:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print classification report
    print("📊 RELATÓRIO DE CLASSIFICAÇÃO:")
    print("=" * 50)
    print(classification_report(y_true_flat, y_pred_flat, target_names=class_names))
    
    return ious, cm

def compare_models_visually(models_results, X_test, Y_test, num_samples=3):
    """Comparar resultados de múltiplos modelos visualmente"""
    fig, axes = plt.subplots(num_samples, len(models_results) + 2, 
                            figsize=(4*(len(models_results) + 2), 4*num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        # Imagem original
        axes[i, 0].imshow(X_test[i])
        axes[i, 0].set_title('Original', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Máscara verdadeira
        axes[i, 1].imshow(Y_test[i], cmap='viridis', vmin=0, vmax=2)
        axes[i, 1].set_title('Ground Truth', fontweight='bold')
        axes[i, 1].axis('off')
        
        # Predições de cada modelo
        for j, (model_name, model_data) in enumerate(models_results.items()):
            if 'model' in model_data:
                model = model_data['model']
                pred = model.predict(X_test[i:i+1])
                pred_class = np.argmax(pred, axis=-1)[0]
                
                axes[i, j+2].imshow(pred_class, cmap='viridis', vmin=0, vmax=2)
                
                # Calcular IoU rápido para esta predição
                intersection = np.logical_and(Y_test[i], pred_class)
                union = np.logical_or(Y_test[i], pred_class)
                iou = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0
                
                axes[i, j+2].set_title(f'{model_name}\\nIoU: {iou:.3f}', fontweight='bold')
                axes[i, j+2].axis('off')
    
    plt.tight_layout()
    plt.show()

def create_comprehensive_report(model, history, X_test, Y_test, model_name="Modelo"):
    """Criar relatório completo do modelo"""
    print(f"📋 RELATÓRIO COMPLETO: {model_name}")
    print("=" * 60)
    
    # 1. Histórico de treinamento
    print("📈 1. HISTÓRICO DE TREINAMENTO:")
    plot_training_history(history, f"{model_name} - Training Progress")
    
    # 2. Predições e visualizações
    print("🎨 2. VISUALIZAÇÕES DE PREDIÇÃO:")
    predictions, pred_classes = predict_and_visualize(model, X_test, Y_test, num_samples=4)
    
    # 3. Análise por classe
    print("📊 3. ANÁLISE POR CLASSE:")
    ious, cm = analyze_class_performance(Y_test, pred_classes)
    
    # 4. Métricas finais
    print("🎯 4. MÉTRICAS FINAIS:")
    final_acc = history.history['val_accuracy'][-1] if 'val_accuracy' in history.history else 0
    final_miou = history.history['val_mean_iou_modular'][-1] if 'val_mean_iou_modular' in history.history else 0
    
    print(f"   • Accuracy Final: {final_acc:.4f} ({final_acc*100:.2f}%)")
    print(f"   • mIoU Final: {final_miou:.4f} ({final_miou*100:.2f}%)")
    print(f"   • IoU Background: {ious[0]:.4f}")
    print(f"   • IoU Cat: {ious[1]:.4f}")
    print(f"   • IoU Dog: {ious[2]:.4f}")
    
    return {
        'final_accuracy': final_acc,
        'final_miou': final_miou,
        'ious_per_class': ious,
        'confusion_matrix': cm,
        'predictions': predictions
    }

print("✅ Sistema de visualização carregado!")
print("🎨 Funções disponíveis:")
print("   • plot_training_history() - Gráficos de treinamento")
print("   • predict_and_visualize() - Predições visuais")
print("   • analyze_class_performance() - Análise por classe")
print("   • compare_models_visually() - Comparação visual")
print("   • create_comprehensive_report() - Relatório completo")

🎨 Carregando sistema de visualização...
✅ Sistema de visualização carregado!
🎨 Funções disponíveis:
   • plot_training_history() - Gráficos de treinamento
   • predict_and_visualize() - Predições visuais
   • analyze_class_performance() - Análise por classe
   • compare_models_visually() - Comparação visual
   • create_comprehensive_report() - Relatório completo


In [383]:
# Treinamento:
model, history = train_model_modular(
    X_train, Y_train, (X_val, Y_val),
    architecture='unet_improved',  # ou 'unet_base', 'unet_attention'
    optimization_config='advanced'
)

🚀 INICIANDO TREINAMENTO MODULAR
🏗️ Criando modelo: unet_improved
⚙️ Compilando modelo...
⚡ Criando otimizador: adam
   Learning Rate: 0.0001
   🎯 Adam: adaptativo, estável, recomendado
✅ Modelo compilado!
📊 Preparando dados...
🔄 Dados normalizados (0-1)
✅ Dados preparados!
   📊 X_train: (172, 256, 256, 3)
   📊 Y_train: (172, 256, 256)
   📊 X_val: (24, 256, 256, 3)
   📊 Y_val: (24, 256, 256)
✅ Callbacks avançados configurados!
📉 ReduceLR: fator=0.5, paciência=4
📊 CSV Log: training_log_modular.csv
🎨 Aplicando advanced augmentation...
📈 Configurações de treinamento:
   🏗️ Arquitetura: unet_improved
   ⚙️ Otimização: advanced
   🎨 Augmentation: advanced
   📊 Batch size: 16
   🔄 Epochs: 25
   🎯 Monitor: val_mean_iou
\n🔥 Iniciando treinamento...
--------------------------------------------------
⚙️ Compilando modelo...
⚡ Criando otimizador: adam
   Learning Rate: 0.0001
   🎯 Adam: adaptativo, estável, recomendado
✅ Modelo compilado!
📊 Preparando dados...
🔄 Dados normalizados (0-1)
✅ Dados pr

KeyboardInterrupt: 